<a href="https://colab.research.google.com/github/megmenegazzi/AMD-project/blob/main/AMD_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Install stuff**

In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 47 kB/s 
     |████████████████████████████████| 199 kB 60.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=7cbd353e7ff957835844b0deb7b55807c462b11637b2e9a6669c6617cc2d5f35
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [12]:
!pip install -q findspark

## **Import stuff**

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [2]:
import pyspark
type(spark)

pyspark.sql.session.SparkSession

In [3]:
sc = spark.sparkContext

In [4]:
import os
import findspark
import pandas as pd

## **Load Data**

In [10]:
os.environ["KAGGLE_USERNAME"] = "margheritamenegazzi"
os.environ["KAGGLE_KEY"] = "75953089094982034e32cf970ea2d0e2"

In [16]:
!kaggle datasets download ashirwadsangwan/imdb-dataset --unzip

 99% 1.43G/1.44G [00:06<00:00, 234MB/s]
100% 1.44G/1.44G [00:06<00:00, 233MB/s]


## **Clean Data**

In [15]:
data = pd.read_csv("title.principals.tsv/title.principals.tsv", sep ="\t")

In [16]:
data.head()

,tconst,ordering,nconst,category,job,characters
0,tt0000001,1,nm1588970,self,\N,"[""Herself""]"
1,tt0000001,2,nm0005690,director,\N,\N
2,tt0000001,3,nm0374658,cinematographer,director of photography,\N
3,tt0000002,1,nm0721526,director,\N,\N
4,tt0000002,2,nm1335271,composer,\N,\N


In [17]:
data.drop(['ordering', 'category', 'job', 'characters'], axis=1, inplace=True)

In [18]:
data.rename(columns={"tconst": "title", "nconst": "actor"}, inplace=True)

In [19]:
data.head()

,title,actor
0,tt0000001,nm1588970
1,tt0000001,nm0005690
2,tt0000001,nm0374658
3,tt0000002,nm0721526
4,tt0000002,nm1335271


## **Load Dataset on Spark**

In [20]:
sc = spark.sparkContext

In [21]:
data = data.to_records(index = False)

In [42]:
simple_rdd = sc.parallelize(data[:100000])

In [45]:
simple_rdd.take(10)

[('tt0000001', 'nm1588970'),
 ('tt0000001', 'nm0005690'),
 ('tt0000001', 'nm0374658'),
 ('tt0000002', 'nm0721526'),
 ('tt0000002', 'nm1335271'),
 ('tt0000003', 'nm0721526'),
 ('tt0000003', 'nm5442194'),
 ('tt0000003', 'nm1335271'),
 ('tt0000003', 'nm5442200'),
 ('tt0000004', 'nm0721526')]

In [50]:
inverted = simple_rdd.map(lambda t : (t[1], t[0]))

In [52]:

joined = (inverted.join(inverted))


In [63]:
filtered = joined.filter(lambda x : x[1][0]!= x[1][1])

In [64]:
links = filtered.map(lambda x : x[1])
#links.take(10)

[('tt0000001', 'tt0000005'),
 ('tt0000001', 'tt0000006'),
 ('tt0000001', 'tt0000007'),
 ('tt0000001', 'tt0000008'),
 ('tt0000001', 'tt0000036'),
 ('tt0000001', 'tt0000060'),
 ('tt0000001', 'tt0000076'),
 ('tt0000001', 'tt0000082'),
 ('tt0000001', 'tt0000108'),
 ('tt0000001', 'tt0000109')]